In [ ]:
import pandas as pd
from sqlalchemy import create_engine

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel

from matplotlib import pyplot as plt
from sklearn.datasets import make_classification
from sqlalchemy import create_engine



In [ ]:
csv_file = "Resources/AQI_Data.csv"
AQI_data_df = pd.read_csv(csv_file)
AQI_data_df.head()

In [ ]:
copy_AQI_data_df